# Filter and Doubling Rate Calculation

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd_raw = pd.read_csv(r'C:\Users\pc\Desktop\time_series_covid19_confirmed_global.csv')
confirmed = pd.DataFrame({'date' : pd_raw.columns[4:]})
confirmed['date'] = confirmed.date.astype('datetime64[ns]')

country_list = pd_raw['Country/Region'].drop_duplicates().tolist()

for each in country_list:
    confirmed[each] = np.array(np.array(pd_raw[pd_raw['Country/Region'] == each].iloc[:,4::].sum(axis = 0)))

confirmed.to_csv(r'C:\Users\pc\Desktop\confirmed.csv', sep = ';', index = False)

In [3]:
from scipy import signal

def savgol_filter(dataframe, window):
    return signal.savgol_filter(np.array(dataframe), window, 1)

from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept = True)

def get_doubling_time_via_regression(array):
    reg.fit(np.arange(-1,2).reshape(-1, 1),np.array(array))
    return reg.intercept_/reg.coef_

def rolling_reg(dataframe):
    return dataframe.rolling(3, 3).apply(get_doubling_time_via_regression)

In [4]:
confirmed_filtered = confirmed.loc[:, confirmed.columns != 'date']
for each in country_list:
    confirmed_filtered[each] = pd.DataFrame(savgol_filter(confirmed_filtered[each], 5))
confirmed_filtered.columns = country_list
confirmed_filtered.insert(0, 'date', confirmed.iloc[: , :1].values)

confirmed_filtered.to_csv(r'C:\Users\pc\Desktop\confirmed_filtered.csv', sep = ';', index = False)

In [5]:
confirmed_DR = pd.DataFrame(rolling_reg(confirmed.loc[:, confirmed.columns != 'date']))
confirmed_DR.replace([np.inf, -np.inf], np.nan, inplace = True)
confirmed_DR.insert(0, 'date', confirmed.iloc[: , :1].values)

confirmed_DR.to_csv(r'C:\Users\pc\Desktop\confirmed_DR.csv', sep = ';', index = False)

In [6]:
mask = confirmed[country_list] > 100
confirmed_filtered_DR = pd.DataFrame(rolling_reg(confirmed_filtered.loc[:, confirmed_filtered.columns != 'date']))
confirmed_filtered_DR.insert(0, 'date', confirmed.iloc[: , :1].values)
confirmed_filtered_DR.replace([np.inf, -np.inf], np.nan, inplace = True)
confirmed_filtered_DR[country_list] = confirmed_filtered_DR[country_list].where(mask)

confirmed_filtered_DR.to_csv(r'C:\Users\pc\Desktop\confirmed_filtered_DR.csv', sep = ';', index = False)

# Visualization

In [7]:
import plotly
import plotly.graph_objects as go

In [8]:
data = pd.DataFrame()

confirmed = pd.read_csv(r'C:\Users\pc\Desktop\confirmed.csv', sep = ';')
confirmed_filtered = pd.read_csv(r'C:\Users\pc\Desktop\confirmed_filtered.csv', sep = ';')
confirmed_DR = pd.read_csv(r'C:\Users\pc\Desktop\confirmed_DR.csv', sep = ';')
confirmed_filtered_DR = pd.read_csv(r'C:\Users\pc\Desktop\confirmed_filtered_DR.csv', sep = ';')

In [9]:
import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()
fig = go.Figure()

font_style = {'font-family': 'Friz Quadrata Std','font-size': 14}

country_options = []
for each in country_list:
    country_options.append({'label': each, 'value': each})    

app.layout = html.Div([

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    ''', style = font_style),
    
    dcc.Dropdown(
        id = 'country_drop_down',
        options = country_options,
        #value = ['US', 'Germany', 'Italy'],
        value = ['Norway', 'Austria', 'Italy'],
        multi = True,
        style = font_style
    ),
       
    dcc.Markdown('''
    ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        ''', style = font_style),
    
    dcc.Dropdown(
        id = 'doubling_time',
        options = [
            {'label': 'Timeline Confirmed', 'value': 'confirmed'},
            {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
            {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
            {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
                ],
        value = 'confirmed',
        multi = False,
        style = font_style
    ),
    
    dcc.RadioItems(
       id = 'scale_option',
       options = [
           {'label': 'Logarthmic Scale', 'value': 'logarthmic scale'},
           {'label': 'Linear Scale', 'value': 'linear scale'},
               ],
        value = 'logarthmic scale',
        style = font_style
    ),
    
    dcc.Graph(figure = fig, id = 'main_window_slope')
])

In [10]:
from dash.dependencies import Input, Output

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value'),
    Input('scale_option', 'value')])
def update_figure(country_list, show_doubling, choose_scale):
    traces = []

    if show_doubling == 'confirmed':
        data = confirmed
    if show_doubling == 'confirmed_filtered':
        data = confirmed_filtered
    if show_doubling == 'confirmed_DR':
        data = confirmed_DR
    if show_doubling == 'confirmed_filtered_DR':
        data = confirmed_filtered_DR
        
    
    for each in country_list:
        traces.append(dict(x = data.date,
                                y = data[each],
                                mode = 'markers+lines',
                                line_width = 1,
                                marker_size = 1, 
                                name = each
                          )
                     )
        
    return {
            'data': traces,
            'layout': dict (
                width = 1366,
                height = 500,
                xaxis = {'title':'Time', 'tickangle':-45, 'nticks':20},
                yaxis = {'title': 'Approximated doubling rate over 3 days'if 'DR' in show_doubling
                         else 'Confirmed infected people (' + choose_scale + ')',
                         'type': 'log' if choose_scale == 'logarthmic scale' else 'linear'},
                font = dict(family = font_style['font-family'], size = font_style['font-size'])
            )
    }

In [ ]:
app.run_server(debug = True, use_reloader = False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
